In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Dec 19 17:08:19 2024

@author: Rodrigo Meira
"""
from libs.eos_database import *
from libs.compressor_class import *
from libs.compression import *
from libs.gc_eos_soave import *
from libs.viscosity import *
from libs.plenum_system import *
from libs.simulation import *

from casadi import *
import numpy as np
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import torch

In [ ]:
list_names = ["CH4", "C2H6", "C3H8", "iC4H10", "nC4H10", "iC5H12", "nC5H12", 
                  "nC6H14", "nC7H16", "nC8H18", "nC9H20", "nC10H22", "nC11H24", 
                   "nC12H26", "nC14H30", "N2", "H2O", "CO2", "C15+"]

nwe = [0.9834, 0.0061, 0.0015, 0.0003, 0.0003, 0.00055, 0.0004, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003, 0.0, 0.0008, 0.0]

dict_composition= {list_names[i]: nwe[i] for i in range(len(nwe))}

mixture = Mixture(list_of_species, dict_composition)

volumn_desviation = [0] * 19

vlv = valve(kv=0.38)
Vpp = 2.0 
Lc = 2.0 
A1 = 2.6e-3
gas = gc_eos_class(mixture, 300, 4500, None, 1, 0, Aij, volumn_desviation, 'gas')
comp = CompressorClass()
visc = viscosity(mixture, volumn_desviation)
compressor = compression(gas, comp, visc)
plenum_sys = plenum(gas, compressor, vlv, Vpp, Lc, A1)
nAlphas = 100
N_RotS = np.random.uniform(600, 750, nAlphas+1)
alphas = np.random.uniform(0.35,0.65, nAlphas+1)
nData = 120
dt = 0.5
timestep = 3
x0 = [14.9919, 339.69, 0.42885]
z0 = [6245.39, 6245.39, 321.672, 0.445562, 319.423, 0.503621, 320.097, 0.396345, 339.69, 0.42885, 0.514917]
u0 = [4500, 300, 600, alphas[0], 5000]

#medido: tp, pp, mp, p2, t2, todos u0.

In [3]:
sim = Simulation(plenum_sys, compressor, x0, z0, u0, nAlphas, alphas, N_RotS, nData, dt, timestep)
x_values, z_values, time_steps, alpha_values, N_values, x_train, y_train, x_min, x_max = sim.run()

CasADi - 2025-03-17 16:00:52 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:515]


In [7]:
print(x_train[0])
print(y_train[0])
print(x_min)

tensor([[8.4303e+00, 3.3545e+02, 7.5627e+03, 1.0575e+04, 3.5532e+02, 4.1808e-01,
         7.2100e+02],
        [8.8154e+00, 3.3564e+02, 7.5726e+03, 1.0479e+04, 3.5497e+02, 4.1808e-01,
         7.2100e+02],
        [9.1844e+00, 3.3588e+02, 7.5869e+03, 1.0354e+04, 3.5452e+02, 4.1808e-01,
         7.2100e+02]])
tensor([[9.5336e+00, 3.3617e+02, 3.4433e-01, 7.6052e+03, 1.0206e+04, 3.2302e+02,
         4.2006e-01, 3.2256e+02, 4.2670e-01, 3.4975e+02, 2.6798e-01, 3.5398e+02,
         2.7249e-01, 5.1492e-01]])
tensor([[[8.4303e+00, 3.3545e+02, 7.5627e+03, 6.4115e+03, 3.3398e+02,
          4.1808e-01, 6.4896e+02]]])


In [ ]:
plt.figure(figsize=(12, 6))

plt.figure(figsize=(12, 6))
plt.plot(time_steps, x_values[:, 0], label=f'Vazao Massica')
plt.xlabel('Tempo')
plt.ylabel(f'Vazao massica')
plt.title(f'Evolução da Vazao massica ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, x_values[:, 1], label=f'Temperatura')
plt.xlabel('Tempo')
plt.ylabel(f'Temperatura')
plt.title(f'Evolução da Temperatura ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, x_values[:, 2], label=f'Volume')
plt.xlabel('Tempo')
plt.ylabel(f'Volume')
plt.title(f'Evolução do Volume ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, z_values[:, 0], label='Pp', color='green')
plt.xlabel('Tempo')
plt.ylabel('Estado z[0]')
plt.title('Evolução da Pressao do Plenum ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, alpha_values, label='Abertura da válvula', color='red')
plt.xlabel('Tempo')
plt.ylabel('Abertura da Válvula')
plt.title('Variação da Abertura da Válvula ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(12, 6))
plt.plot(time_steps, N_values, label='Rotacao', color='red')
plt.xlabel('Tempo')
plt.ylabel('Rotacao')
plt.title('Variação da Rotacao ao Longo do Tempo')
plt.legend()
plt.grid()
plt.show()